In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import cv2
import os
import time

import torch
import torchvision.ops.boxes # to calculate iou score

In [2]:
# list of available trackers
# Only 'DASIAMRPN' and 'GOTURN' are DL-based
tracker_lst = ['BOOSTING', 'CSRT', 'DASIAMRPN', 'GOTURN', 'KCF', 'MEDIANFLOW', 'MIL', 'MOSSE', 'TLD']
tracker_lst = ['DASIAMRPN', 'GOTURN']

# see how different trackers perform on the Fish sequence
seq_name = 'jogging'

In [3]:
def get_num_of_frames(seq_name):
    frames = []
    seq_img_folder_path = './sequences/' + seq_name + '/img'
    seq_img_folder = os.listdir(seq_img_folder_path)
    for img_name in seq_img_folder:
        if img_name.endswith('.jpg'):
            frames.append(img_name)
    return len(frames)

def get_first_frame_path(seq_name):
    frame_path = ''
    seq_img_folder_path = './sequences/' + seq_name + '/img'
    seq_img_folder = os.listdir(seq_img_folder_path)
    for img_name in seq_img_folder:
        if img_name.endswith('.jpg'):
            frame_path = seq_img_folder_path + '/' + img_name
            break
    return frame_path

def get_frame_size(seq_name):
    width = 0
    height = 0
    first_frame = []
    seq_img_folder_path = './sequences/' + seq_name + '/img'
    seq_img_folder = os.listdir(seq_img_folder_path)
    first_frame = cv2.imread(get_first_frame_path(seq_name))

    height, width, channel = first_frame.shape
    return width, height

In [4]:
df_seq_info = pd.DataFrame(0, columns=['num_of_frames', 'frame_width', 'frame_height'], index=[seq_name])

for seq_idx, seq_row in df_seq_info.iterrows():
    num_of_frames = get_num_of_frames(seq_idx)
    frame_width, frame_height = get_frame_size(seq_idx)
    df_seq_info.at[seq_idx, 'num_of_frames'] = num_of_frames
    df_seq_info.at[seq_idx, 'frame_width'] = frame_width
    df_seq_info.at[seq_idx, 'frame_height'] = frame_height
    
df_seq_info

,num_of_frames,frame_width,frame_height
jogging,307,352,288


In [5]:
# FPS: average frames per second
# Fail: number of failed frames
# IoU: Intersection over Union

df_results = pd.DataFrame(0, columns=['FPS (average)', 'Fail (count)', 'IoU (ratio)'], index=tracker_lst)

In [6]:
for tracker_name in tracker_lst:
    result_path = "./results/" + seq_name + "/" + tracker_name + "/"
    
    # calculate average fps
    fps_file = open(result_path+"fps.txt", "r")
    fps_lst = []
    for line in fps_file:
        fps_lst.append(int(line.rstrip()))
    df_results.loc[tracker_name, 'FPS (average)'] = round(sum(fps_lst) / len(fps_lst), 2)
    
    # obtain number of failed frames and IoU
    bbox_tracked_file = open(result_path+"bbox_tracked_object1.txt", "r")
    num_fail = 0
    bbox_tracked_lst = []
    bbox_truth_lst = []
    iou_lst = []
    
    ## get the tracked (predicted) bounding boxes
    for line in bbox_tracked_file:
        temp_str = line.rstrip()
        if temp_str == "failed":
            num_fail += 1
            bbox_tracked_lst.append([0, 0, 0, 0])
        else:
            temp_bbox = temp_str.split(', ') # in format [x, y, width, height]
            bbox_tracked_lst.append([
                int(temp_bbox[0]),
                int(temp_bbox[1]),
                int(temp_bbox[0]) + int(temp_bbox[2]),
                int(temp_bbox[1]) + int(temp_bbox[3])
            ]) # change to format [x_left, y_top, x_right, y_bottom]
    bbox_truth_file = open("./sequences/" + seq_name + "/" + "groundtruth_rect.txt", "r")
    
    ## get the true bounding boxes
    for line in bbox_truth_file:
        temp_bbox = line.rstrip().split('\t') # in format [x, y, width, height]
        bbox_truth_lst.append([
                int(temp_bbox[0]),
                int(temp_bbox[1]),
                int(temp_bbox[0]) + int(temp_bbox[2]),
                int(temp_bbox[1]) + int(temp_bbox[3])
            ]) # change to format [x_left, y_top, x_right, y_bottom]
        
    ## compute the iou of each frame
    for bbox_idx in range(len(bbox_tracked_lst)):
        box_tracked = torch.tensor([bbox_tracked_lst[bbox_idx]], dtype=torch.float)
        box_truth = torch.tensor([bbox_truth_lst[bbox_idx+1]], dtype=torch.float) # +1 due to first frame
        iou_lst.append(torchvision.ops.boxes.box_iou(box_tracked, box_truth).item())
    
    # update fail count and iou
    df_results.loc[tracker_name, 'Fail (count)'] = num_fail
    df_results.loc[tracker_name, 'IoU (ratio)'] = round(sum(iou_lst) / len(iou_lst), 4)
    

In [7]:
df_results

,FPS (average),Fail (count),IoU (ratio)
DASIAMRPN,17.35,0,0.6952
GOTURN,16.10,0,0.2344
